# Preprocessing of file "df_clean_v2.gzip" 
The file contains all movies with filled genre_id and poster_url.
This notebook will do the following preprocessing:
* Converting genre_id to datatype list
* Renaming column "genre_ids2" to "genre_txt"
* Adding column "filename"
* Doing a genre_id categorigal encoding (Hot-Encoding)
* Deleting movies without release-date
* Deleting movies wihout an exising poster file
* Writing of a selection of columns to output file

In [39]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
import pyarrow
import glob
import urllib.parse as up
import os

import pyarrow.parquet as pq
import re

import src.helper.const as const
import src.helper.helper as hlp

In [47]:
# Config
CHECK_FILE_EXISTENCE = True

FILE_DIR = const.FILE_DIR
DATA_DIR_INTERIM = const.DATA_DIR_INTERIM
DATA_DIR_RAW = const.DATA_DIR_RAW
POSTER_DIR = const.POSTER_DIR

VERSION = "v3"
FN_DF_IN = DATA_DIR_INTERIM + f"df_cleaned_{VERSION}.gzip"
FN_DF_OUT = DATA_DIR_INTERIM + f"df_multilabel_{VERSION}.gzip"

In [53]:
# Init
pd.set_option('display.max_colwidth', None)
MOVIE_COLS_OUT = ['id',
              'original_title',
              'release_date',
              'popularity',
              'adult',
              'video',
              'poster_url',
              'poster_exists',
              'filename',
              'genre_id',
              'genre_ids2',
              'genre_ids2_list',
              'genre_id_count']
GENRE_OHE_COLS = const.GENRE_OHE_COLS
OUT_COLS = MOVIE_COLS_OUT + GENRE_OHE_COLS

In [72]:
# Load file to process
df = pd.read_parquet(FN_DF_IN)

# Reset index, file index not in a row
df = df.reset_index(drop=True)

In [73]:
# Remove movies containing no release-date
print(f"Number of movies before release-date correction: {len(df)}")
df = df.dropna(subset=['release_date'])
print(f"Number of movies after release-date correction:  {len(df)}")

# Reset index
df = df.reset_index(drop=True)

# Calc number of genre per movie
df = df.drop(columns=["genre_id_count"])
df["genre_id_count"] = [len(np.fromstring(re.sub("\[|\]| ", "", e), dtype=int, sep=",")) for e in list(df.genre_id)]

# Converting genre_id to list
df['genre_id'] = df['genre_id'].apply(lambda x: eval(x))

# Extract filename from poster_url
def retrieveFilename(url):
    a = up.urlparse(url)
    picfilename = os.path.basename(a.path)
    return picfilename

df["filename"] = df.loc[~df["poster_url"].isnull(),"poster_url"].apply(retrieveFilename)

# Check for poster existence
if CHECK_FILE_EXISTENCE:
    def check_image_exists(df):
        '''Checks the presence of a filename and fills a column regarding
        '''
        l_files = os.listdir(POSTER_DIR)
        l_check_res = []
        for idx, row in df.iterrows():
            pfname = re.findall("\w+.jpg", row.poster_url)
            l_check_res.append(pfname[0] in l_files)
        return pd.DataFrame(l_check_res, columns=["poster_exists"])

    # Write result to column "poster_exists"
    print(f"Number of movies before image-check correction: {len(df)}")
    df = df.drop(columns=["poster_exists"])
    df_res = hlp.parallelize_dataframe(df, check_image_exists)
    df["poster_exists"] = df_res["poster_exists"].to_list()
    df = df[df.poster_exists == True]
    print(f"Number of movies after image-check correction: {len(df)}")

# Tramsform "genre_txt" from text to list
l_ml = []
for i,r in df.iterrows():
    l_ml.append(re.split(",", r["genre_ids2"][0]))
df["genre_ids2_list"] = l_ml

# Create multi hot-encoded multi labels on column "genre_txt_list"
mlb = MultiLabelBinarizer()
df_ml = pd.DataFrame(mlb.fit_transform(df['genre_ids2_list']), columns=mlb.classes)
df_ml = df_ml.reset_index(drop=True)

# Show labels
print(mlb.classes_)

# Add hot-encoded labels to dataframe
df_ml.columns = mlb.classes_
df = df.join(df_ml)

# Check it
df[['genre_ids2'] + GENRE_OHE_COLS]

In [77]:
# Show missing values, only url is allowed to be empty
print("Missing values:\n{df.isna().sum()}")

print("Genre distribution:")
df_dist = pd.DataFrame(df.genre_id_count.value_counts().sort_index().reset_index())
df_dist.columns=["Number of genres per movie", "Number of movies"]
df_dist = df_dist.set_index("Number of genres per movie")
df_dist

Missing values:
{df.isna().sum()}
Genre distribution:


,Number of movies
Number of genres per movie,
1,151966
2,77790
3,34066
4,8597
5,1973
6,389
7,60
8,11
9,2


In [78]:
# Save result
df[OUT_COLS].to_parquet(FN_DF_OUT, compression='gzip')